## Parse The Questions

In [6]:
from httpx import get
quiz_notes = get('https://gist.githubusercontent.com/hamelsmu/61f591a5770e49421a4fc69dd966b16a/raw/e8c73442cbeccc9a08a81504631c4bb11409475e/claude_conversation.md').text

We want to take the 30 quiz questions from the conversation in the local file `claude_conversation.md` and create a repo with a with a directory for each question.  Here are the contents of this file:

In [10]:
!cat claude_conversation.md

# HTMX Mastery Quiz
## 🧑 human _(Sep 28, 2024, 07:53 PM)_
Create a series of questions, including code challenges that test a readers understanding of this material on HTMX:  

Aim for 30 or so questions covering these topics.  First, generate an outline of the specific topics you will create quiz questions about.  The goal of these questions is to push the reader to achieve technical mastery of the subject.
```
Previous: A Dynamic Archive UI

Next: Client Side Scripting

# Tricks Of The Htmx Masters

Contents

  * Htmx Attributes
    * hx-swap
    * hx-trigger
      * Trigger filters
      * Synthetic events
    * Other Attributes
  * Events
    * Htmx-Generated Events
    * Using the htmx:configRequest Event
    * Canceling a Request Using htmx:abort
    * Server Generated Events
  * HTTP Requests & Responses
    * HTTP Response Codes
  * Updating Other Content
    * Expanding Your Selection
    * Out of Band Swaps
    * Events
    * Being Pragmatic
  * Debugging
    * Logging Htmx E

In [18]:
%%ai

First, we want to parse `claude_conversation.md` and extract each of the 30 questions from the section that has more specific, code-based questions, into a python list. Make sure preserve any bits of code and other parts of the question that are useful in the content of the list.

##### Assistant

Certainly! I'll parse the `claude_conversation.md` file and extract the 30 code-based questions into a Python list. Here's the code to do that:

In [19]:
import re

questions = []
with open('claude_conversation.md', 'r') as file:
    content = file.read()

# Find the section with code-based questions
start = content.index("1. Given the following HTML, modify the `hx-swap` attribute")
end = content.index("These code-based questions should provide a more hands-on test")

# Extract the questions
question_section = content[start:end]
pattern = r'\d+\.\s(.*?)(?=\d+\.\s|\Z)'
matches = re.findall(pattern, question_section, re.DOTALL)

questions = [q.strip() for q in matches]

print(f"Extracted {len(questions)} questions.")
print("First question:", questions[0])
print("Last question:", questions[-1])

Extracted 30 questions.
First question: Given the following HTML, modify the `hx-swap` attribute to fade out the old content over 300ms before swapping in the new content:

```html
<div hx-get="/data" hx-swap="innerHTML">Load Data</div>
```
Last question: Modify the following HTML to swap in new content every 30 seconds, but stop after 5 swaps:

```html
<div hx-get="/live-data" hx-trigger="every 30s">
  <!-- Initial content here -->
</div>
```


This code will extract the questions into the `questions` list. Each item in the list will contain the full text of the question, including any code snippets or additional context.

You can now use this `questions` list for further processing or to create the quiz structure. Would you like me to proceed with creating the directory structure for the quiz based on these questions?

##### User

In [24]:
print(questions[3])

Complete the JavaScript to add a custom "X-CSRF-Token" header to all HTMX requests:

```javascript
document.body.addEventListener('htmx:configRequest', (event) => {
  // Add your code here
});
```


In [25]:
%%aip

save `questions` to disk.

In [26]:
import pickle

with open('htmx_questions.pkl', 'wb') as f:
    pickle.dump(questions, f)

# To verify the save was successful, we can try to load it back
with open('htmx_questions.pkl', 'rb') as f:
    loaded_questions = pickle.load(f)

# Check if the loaded questions match the original
print(f"Save successful: {loaded_questions == questions}")
print(f"Number of questions saved: {len(loaded_questions)}")

Save successful: True
Number of questions saved: 30


In [28]:
%ai reset

# Generate Directory Structure

In [29]:
import ContextKit as ck
import claudette as ct
with open('htmx_questions.pkl', 'rb') as f: questions = pickle.load(f)
fh_docs = ck.fasthtml()
ct_docs = ck.claudette()

In [30]:
%%ai

You have a list of questions about HTMX stored in `questions`.  Use claudette as illustrated in $`ct_docs` and construct a prompt that will allow you to do the following for each HTMX question:

1. Create a short directory name with a number prefix. 
2. Create a README in that directory with the question (this part doesn't need claudette, you can just copy the question to the README.)

##### Assistant

To accomplish this task, I'll use claudette to generate a structured directory listing for the HTMX questions. Here's how we'll do it:

1. Create a Chat instance with a custom system prompt.
2. Define a function to process each question.
3. Use toolloop to generate the directory structure.

Let's start by setting up the Chat and defining our function:

In [32]:
import os

sp = """You are an assistant helping to create a directory structure for HTMX questions. 
For each question, provide a short, numbered directory name (e.g. '01_htmx_basics') and the full path to a README.md file in that directory.
Respond with a Python dict containing 'dir_name' and 'readme_path' keys."""

chat = ct.Chat(ct.models[-1], sp=sp)

def process_question(question):
    prompt = f"Create a directory structure for this HTMX question: {question}"
    response = chat.toolloop(prompt)
    result = eval(response.content[0].text)
    return result

# Process all questions
for i, question in enumerate(questions, 1):
    result = process_question(question)
    
    # Create directory
    os.makedirs(result['dir_name'], exist_ok=True)
    
    # Create README.md with the question
    with open(result['readme_path'], 'w') as f:
        f.write(question)
    
    print(f"{i}. Created: {result['dir_name']}")
    print(f"   README: {result['readme_path']}\n")

1. Created: 01_htmx_swap_fade
   README: 01_htmx_swap_fade/README.md

2. Created: 02_htmx_debounce_input
   README: 02_htmx_debounce_input/README.md

3. Created: 03_htmx_one_time_click
   README: 03_htmx_one_time_click/README.md

4. Created: 04_htmx_csrf_token
   README: 04_htmx_csrf_token/README.md

5. Created: 05_htmx_server_events
   README: 05_htmx_server_events/README.md

6. Created: 06_htmx_abort_request
   README: 06_htmx_abort_request/README.md

7. Created: 07_htmx_delete_response
   README: 07_htmx_delete_response/README.md

8. Created: 08_htmx_out_of_band_swap
   README: 08_htmx_out_of_band_swap/README.md

9. Created: 09_htmx_logging
   README: 09_htmx_logging/README.md

10. Created: 10_htmx_chrome_devtools
   README: 10_htmx_chrome_devtools/README.md

11. Created: 11_htmx_disable_processing
   README: 11_htmx_disable_processing/README.md

12. Created: 12_htmx_global_config
   README: 12_htmx_global_config/README.md

13. Created: 13_htmx_synchronize_requests
   README: 13_htm

This code will process each question and print out the generated directory structure along with the original question. You can then use this information to create the actual directories and README files.

To create the directories and README files, you can add the following code after processing each question:

This will create the directory structure and README files based on the generated names and paths.

## Generate Answers to Quiz Questions in `main.py`

##### User

In [44]:
%%ai

Now, for each directory you created above, use claudette to create a `main.py` in each directory that has minimal FastHTML code to help you get started.  FastHTML code is contained in the variable `fh_docs`, which you should put into the context of your LLM prompt. Make sure you reference the `README` as well in each directory.  Allow the language model to have two sections of outputs in `<explain>` and `<code>` xml tags.  Parse the contents of the `<code>` tags and put that in `main.py` file.  If the contents of `<code>` contain a markdown code block, make sure you parse the code out of that.

Certainly! I'll use claudette to create a `main.py` file in each directory with minimal FastHTML code, referencing the `README.md` and using the FastHTML documentation from `fh_docs`. Here's how we'll do it:

In [46]:
import os

sp = f"""You are an assistant helping to create minimal FastHTML code for HTMX questions.
Read the question from the README.md file and create a basic FastHTML application that addresses the question.
Use the following FastHTML documentation for reference:

{fh_docs}

Provide your response in two sections:
1. <explain>: A brief explanation of the code and how it addresses the question.
2. <code>: The FastHTML code for the main.py file.
"""

chat = ct.Chat(ct.models[-1], sp=sp)

def create_main_py(directory):
    readme_path = os.path.join(directory, 'README.md')
    with open(readme_path, 'r') as f:
        question = f.read()

    prompt = f"Create a minimal FastHTML application for this HTMX question: {question}"
    response = chat.toolloop(prompt)

    explanation = ""
    code = ""
    for block in response.content:
        if hasattr(block, 'text'):
            if '<explain>' in block.text:
                explanation = block.text.split('<explain>')[1].split('</explain>')[0].strip()
            if '<code>' in block.text:
                code = block.text.split('<code>')[1].split('</code>')[0].strip()

    # Remove markdown code block if present
    if code.startswith('```python'):
        code = code.split('\n', 1)[1]
    if code.endswith('```'):
        code = code.rsplit('\n', 1)[0]

    main_py_path = os.path.join(directory, 'main.py')
    with open(main_py_path, 'w') as f:
        f.write(code)

    print(f"Created main.py in {directory}")
    print()

# Process all directories
for directory in sorted(os.listdir('.')):
    if os.path.isdir(directory) and directory.startswith(tuple('0123456789')):
        create_main_py(directory)

Created main.py in 01_htmx_swap_fade

Created main.py in 02_htmx_debounce_input

Created main.py in 03_htmx_one_time_click

Created main.py in 04_htmx_csrf_token

Created main.py in 05_htmx_server_events

Created main.py in 06_htmx_abort_request

Created main.py in 07_htmx_delete_response

Created main.py in 08_htmx_out_of_band_swap

Created main.py in 09_htmx_logging

Created main.py in 10_htmx_chrome_devtools

Created main.py in 11_htmx_disable_processing

Created main.py in 12_htmx_global_config

Created main.py in 13_htmx_synchronize_requests

Created main.py in 14_htmx_trigger_validation

Created main.py in 15_htmx_lazy_loading

Created main.py in 16_htmx_error_handling

Created main.py in 17_htmx_preserve_input

Created main.py in 18_htmx_update_url

Created main.py in 19_htmx_auto_save

Created main.py in 20_htmx_event_handling

Created main.py in 21_htmx_request_queueing

Created main.py in 22_htmx_force_refresh

Created main.py in 23_htmx_loading_indicator

Created main.py in 

This script will:

1. Define a system prompt that includes the FastHTML documentation.
2. Create a function `create_main_py` that:
   - Reads the question from the README.md file.
   - Uses claudette to generate a minimal FastHTML application.
   - Extracts the explanation and code from the response.
   - Writes the code to a `main.py` file in the directory.
3. Iterate through all numbered directories and create a `main.py` file for each.

Let's run this script to create the `main.py` files:

```python
# Run the script to create main.py files
create_main_py_files()
```

This will create a `main.py` file in each directory, containing minimal FastHTML code that addresses the HTMX question from the README.md file. The script will also print out a brief explanation for each file created.